

---



#Preparing

##Just to get acess to the image pattern file

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Heading to workspace

In [ ]:
%cd /content/drive/My Drive/GPX/

##Not by default on Colab

In [ ]:
!pip install osmnx -q
!pip install gpxpy -q

##Loading required modules

In [ ]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import osmnx as ox
from skimage.morphology import skeletonize
import gpxpy
import gpxpy.gpx
from datetime import datetime
from scipy import ndimage
from skimage import measure
import os



---



#this gets city's street network from OpenStreetMap

In [ ]:
place_name = "Vancouver, BC, Canada"
network_type = 'bike'

In [ ]:
print(f"Downloading {network_type} network for {place_name}")
G = ox.graph_from_place(place_name, network_type=network_type)

In [ ]:
print(f"Street network for {place_name}")
fig, ax = ox.plot_graph(G, node_size=0, edge_color='black', edge_linewidth=0.5, bgcolor='white')
plt.show()

In [ ]:
plt.close(fig)



---



#processing image

In [ ]:
filename = 'AzadiSq.png'
filename_without_ext = os.path.splitext(filename)[0]

In [ ]:
try:
  img = Image.open(filename).convert('RGB')
  img_np = np.array(img)
  hsv_img = cv2.cvtColor(img_np, cv2.COLOR_RGB2HSV)
  #
except FileNotFoundError:
  print(f"Error: {filename} not found.")

In [ ]:
plt.figure()
plt.imshow(img_np)
plt.axis('off')

In [ ]:
plt.close()

###color range hsv (update to be inclusive if needed)

In [ ]:
color_ranges = {
            'blue': ([100, 150, 0], [140, 255, 255]),
            'red': ([0, 100, 20], [10, 255, 255], [160, 100, 20], [180, 255, 255]), # Red wraps around
            'green': ([40, 40, 40], [80, 255, 255]),
            'yellow': ([20, 100, 100], [30, 255, 255]),
            'orange': ([10, 100, 20], [25, 255, 255]),
            'purple': ([130, 100, 0], [170, 255, 255]),
            'cyan': ([80, 100, 0], [100, 255, 255]),
            'magenta': ([140, 100, 0], [160, 255, 255]),
            'brown': ([10, 100, 20], [20, 255, 200]), # Approximated range
            'black': None, # Special case for black (handled with grayscale thresholding)
            'white': None, # Special case for white (handled with grayscale thresholding)
        }
#
color_map_rgba = {
  'red': [255, 0, 0, 255],       # Opaque Red
  'blue': [0, 0, 255, 255],      # Opaque Blue
  'green': [0, 255, 0, 255],     # Opaque Green
  'yellow': [255, 255, 0, 255],  # Opaque Yellow
  'orange': [255, 165, 0, 255],  # Opaque Orange
  'purple': [128, 0, 128, 255],  # Opaque Purple
  'cyan': [0, 255, 255, 255],    # Opaque Cyan
  'magenta': [255, 0, 255, 255], # Opaque Magenta
  'brown': [165, 42, 42, 255],   # Opaque Brown
  'black': [0, 0, 0, 255],       # Opaque Black
  'white': [255, 255, 255, 255]  # Opaque White
}

###detects colors used in the image

In [ ]:
detected_colors = []
total_pixels = img_np.shape[0] * img_np.shape[1]
min_pixels_threshold = total_pixels * 0.001 #

for color, ranges in color_ranges.items():
  if color in ['black', 'white']:
    gray_img = img.convert('L')
    gray_img_np = np.array(gray_img)
    if color == 'black':
        ret, mask = cv2.threshold(gray_img_np, 10, 255, cv2.THRESH_BINARY_INV)
    else: # color == 'white'
        ret, mask = cv2.threshold(gray_img_np, 245, 255, cv2.THRESH_BINARY)
  elif color == 'red':
    mask1 = cv2.inRange(hsv_img, np.array(ranges[0]), np.array(ranges[1]))
    mask2 = cv2.inRange(hsv_img, np.array(ranges[2]), np.array(ranges[3]))
    mask = mask1 + mask2
  else:
    lower, upper = ranges
    mask = cv2.inRange(hsv_img, np.array(lower), np.array(upper))
  pixel_count = np.sum(mask > 0)
  if pixel_count > min_pixels_threshold:
      detected_colors.append(color)

In [ ]:
print(f"Detected colors in the image: {detected_colors}")
#
if 'white' in detected_colors: detected_colors.remove('white')
if 'black' in detected_colors: detected_colors.remove('black')
#
print(f"[cleaned] Detected colors in the image: {detected_colors}")

####**specify the outermost color**

In [ ]:
outermost_color = 'yellow'

In [ ]:
detected_colors.remove(outermost_color)
detected_colors.insert(0, outermost_color)
print(f"[sorted] Detected colors in the image: {detected_colors}")

##digitizes colors in image and prepares image binary format

In [ ]:
masks = {}
for color in detected_colors:
  if color.lower() in color_ranges:
    if color == 'red':
      ranges = color_ranges[color]
      mask1 = cv2.inRange(hsv_img, np.array(ranges[0]), np.array(ranges[1]))
      mask2 = cv2.inRange(hsv_img, np.array(ranges[2]), np.array(ranges[3]))
      mask = mask1 + mask2
    else:
      lower, upper = color_ranges[color]
      mask = cv2.inRange(hsv_img, np.array(lower), np.array(upper))
    masks[color] = mask
  else:
    print(f"Warning: color '{color}' not supported - not in color_ranges.")

###visualizing

In [ ]:
for color in detected_colors:
  binary_img = masks[color]
  plt.figure()
  plt.subplot(1, 2, 1)
  plt.imshow(img, cmap='gray')
  plt.title('Original Image')
  plt.axis('off')
  #
  plt.subplot(1, 2, 2)
  plt.imshow(binary_img, cmap='gray')
  plt.title(f'Binary Image ({color})')
  plt.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
plt.close()

##creates outlines for each color

In [ ]:
outline_imgs = {}
#
for color in detected_colors:
  #
  binary_img = masks[color]
  #
  kernel_clean = np.ones((3,3), np.uint8)
  binary_cleaned = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel_clean)
  binary_cleaned = cv2.morphologyEx(binary_cleaned, cv2.MORPH_OPEN, kernel_clean)
  #
  contours, _ = cv2.findContours(binary_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  #
  if not contours:
    print(f'No contours found for color {color}!')
    outline_imgs[color] = None
  else:
    largest_contour = max(contours, key=cv2.contourArea)
    #
    outline_img = np.zeros_like(binary_img)
    cv2.drawContours(outline_img, [largest_contour], -1, 255, 1)  # thickness=1 for thin boundary
    outline_imgs[color] = outline_img

##attempts to get the boundary/perimeter of the pattern as a continuous path

In [ ]:
skeleton_imgs = {}
#
for color in detected_colors:
  kernel = np.ones((2,2), np.uint8)
  skeleton_img = cv2.morphologyEx(outline_imgs[color], cv2.MORPH_CLOSE, kernel)
  skeleton_imgs[color] = skeleton_img

###visualizing

In [ ]:
for color in detected_colors:
  binary_img = masks[color]
  skeleton_img = skeleton_imgs[color]
  #
  plt.figure()
  plt.subplot(1, 3, 1)
  plt.imshow(img, cmap='gray')
  plt.title('Original Image')
  plt.axis('off')
  #
  plt.subplot(1, 3, 2)
  plt.imshow(binary_img, cmap='gray')
  plt.title(f'Binary Image ({color})')
  plt.axis('off')
  #
  plt.subplot(1, 3, 3)
  plt.imshow(skeleton_img, cmap='gray')
  plt.title(f'Boundary Route ({color})')
  plt.axis('off')
  plt.tight_layout()
  plt.show()

In [ ]:
plt.close()



---



#Overlaying image onto map - to detect the main extent using color or outermost layer

##Trial and Error step

**Update these until the outermost boundary sits within the network**

In [ ]:
scale_factor_y = 0.5 # 60% of whole map
scale_factor_x = 0.95 # 60% of whole map
position_offset_x = 0.10 # % shift left bottomright corner image #% of map
position_offset_y = 0.10 # % shift upward bottomright corner image #% of map

In [ ]:
threshold_val = 10
#
existing_extent = None
reference_center = None

In [ ]:
rgba_imgs = {}
image_data = {}

In [ ]:
fig, ax = ox.plot_graph(G, node_size=0, edge_color='black',
                        edge_linewidth=1, bgcolor='white', show=False, close=False)
#
# reference center and all extents
for i, color in enumerate(detected_colors):
    binary_img = masks[color]
    skeleton_img = skeleton_imgs[color]
    thisimage = skeleton_imgs[color]
    #
    if existing_extent:
        graph_xlim = (existing_extent[0], existing_extent[1])
        graph_ylim = (existing_extent[2], existing_extent[3])
    else:
        graph_xlim = ax.get_xlim()
        graph_ylim = ax.get_ylim()
    #
    graph_width = graph_xlim[1] - graph_xlim[0]
    graph_height = graph_ylim[1] - graph_ylim[0]
    #
    img_height, img_width = thisimage.shape
    #
    max_route_width = graph_width * scale_factor_x
    max_route_height = graph_height * scale_factor_y
    #
    width_scale = max_route_width / img_width
    height_scale = max_route_height / img_height
    final_scale = min(width_scale, height_scale)
    #
    new_width = img_width * final_scale
    new_height = img_height * final_scale
    #
    image_data[color] = {
        'image': thisimage,
        'new_width': new_width,
        'new_height': new_height,
        'final_scale': final_scale,
        'img_height': img_height,
        'img_width': img_width
    }
    #
    # center from first image
    if reference_center is None:
        offset_x, offset_y = position_offset_x, position_offset_y
        route_right = graph_xlim[1] - (graph_width * offset_x)
        route_left = route_right - new_width
        route_bottom = graph_ylim[0] + (graph_height * offset_y)
        route_top = route_bottom + new_height
        #
        reference_center = (
            (route_left + route_right) / 2,  # center_x
            (route_bottom + route_top) / 2   # center_y
        )
#
# overlay all images centered on reference
for color in detected_colors:
    data = image_data[color]
    thisimage = data['image']
    new_width = data['new_width']
    new_height = data['new_height']
    img_height = data['img_height']
    img_width = data['img_width']
    #
    center_x, center_y = reference_center
    #
    route_left = center_x - new_width / 2
    route_right = center_x + new_width / 2
    route_bottom = center_y - new_height / 2
    route_top = center_y + new_height / 2
    #
    rgba_img = np.zeros((img_height, img_width, 4), dtype=np.uint8)
    rgba_img[thisimage > 0] = color_map_rgba.get(color.lower(), [255, 0, 0, 255])
    #
    extent = (route_left, route_right, route_bottom, route_top)
    #
    ax.imshow(rgba_img, extent=extent, alpha=0.4, zorder=100)
    #
    kernel = np.ones((5, 5), np.uint8)
    thicker_img = cv2.dilate(rgba_img, kernel, iterations=2)
    ax.imshow(thicker_img, extent=extent, alpha=0.4, zorder=100)
    #
    if existing_extent is None:
        existing_extent = extent
    rgba_imgs[color] = rgba_img
    #
plt.title(f"pattern as routes mapped to city street network", fontsize=16)
plt.show()

> **Does it look good?**

> **Does it fit in the network?**



---



##Production step

In [ ]:
coordinates = {}

In [ ]:
figv, axv = ox.plot_graph(G, node_size=0, edge_color='black',
                        edge_linewidth=0.5, bgcolor='white', show=False, close=False)
#
for _, color in enumerate(detected_colors):
  rgba_img = rgba_imgs[color]
  #
  left, right, bottom, top = extent
  height, width, _ = rgba_img.shape
  #
  lon_per_pixel = (right - left) / width
  lat_per_pixel = (top - bottom) / height
  #
  route_pixels = []
  #
  for y in range(height):
      for x in range(width):
          if np.array_equal(rgba_img[y, x], color_map_rgba.get(color.lower(), [255, 0, 0, 255])):
              route_pixels.append((x, y))
  #
  route_pixels = np.array(route_pixels)
  #
  start_idx = np.argmin(route_pixels[:, 0])
  ordered_pixels = [route_pixels[start_idx]]
  remaining = list(range(len(route_pixels)))
  remaining.remove(start_idx)
  #
  current_pixel = route_pixels[start_idx]
  #
  while remaining:
      distances = []
      for idx in remaining:
          pixel = route_pixels[idx]
          #
          dist = np.sqrt((current_pixel[0] - pixel[0])**2 + (current_pixel[1] - pixel[1])**2)
          distances.append((dist, idx))
      #
      distances.sort()
      #
      chosen_idx = None
      for dist, idx in distances:
          if dist <= 2.0: # Consider pixels within a 2-pixel radius
              chosen_idx = idx
              break
      if chosen_idx is None:
          # If no pixel is within the radius, take the globally closest one
          chosen_idx = distances[0][1]
      current_pixel = route_pixels[chosen_idx]
      ordered_pixels.append(current_pixel)
      remaining.remove(chosen_idx)
  coords = []
  for x, y in ordered_pixels:
      # Convert pixel coordinates to GPS coordinates
      latitude = top - (y + 0.5) * lat_per_pixel
      longitude = left + (x + 0.5) * lon_per_pixel
      coords.append((latitude, longitude))
  coordinates[color] = coords
  #
  if coords:
    lats = [coord[0] for coord in coords]
    lons = [coord[1] for coord in coords]
    #
    axv.scatter(lons, lats, c=color, s=2, alpha=0.7, label=f'Route ({len(coords)} points)')
    #
    axv.plot(lons, lats, color, linewidth=1, alpha=0.6)
    #
axv.set_title("final route - GPS coordinates on city map", fontsize=16)
axv.legend()
plt.show()



---



##Writing out

#Creating GPS and exporting GPX file

In [ ]:
for _, color in enumerate(detected_colors):
  #
  filename_gpx=filename_without_ext+'_'+color
  timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
  output_filename = f"{filename_gpx}_{timestamp}.gpx"
  #
  coords = coordinates[color]
  #
  if not coords:
      print("No coordinates to save!")
  #
  gpx = gpxpy.gpx.GPX()
  gpx_track = gpxpy.gpx.GPXTrack()
  gpx.tracks.append(gpx_track)
  gpx_segment = gpxpy.gpx.GPXTrackSegment()
  gpx_track.segments.append(gpx_segment)
  #
  for lat, lon in coords:
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(lat, lon))
  #
  with open(output_filename, 'w') as f:
    f.write(gpx.to_xml())
  print(f"GPX route for color {color} saved to: {output_filename}")



---

